# Trusted zone

In [93]:
# ! pip install duckdb --user --upgrade --quiet
# ! pip install pyarrow --user --upgrade --quiet

In [94]:
import duckdb
import numpy as np
import pandas as pd

## Joining tables per data source

### household

In [95]:
def join_household():
    con = duckdb.connect('household.duckdb')
    con.execute("CREATE TABLE household AS SELECT * FROM household2018").df()
    con.execute("INSERT INTO household SELECT * FROM household2019").df()
    con.execute("INSERT INTO household SELECT * FROM household2020").df()
    view_household = con.execute(f'SELECT * FROM household').df()
    display(view_household)
    con.close()


### nationalities

In [96]:
def createNatTable(dictColNames):
    sql_create = 'CREATE TABLE nationalities( '
    
    for key in dictColNames:
        sql_create += f'{key} {dictColNames[key]}, ' 
    sql_create += ')'
   
    con = duckdb.connect('nationalities.duckdb')
    con.execute(sql_create)
    con.close()
    return dictColNames
      


In [97]:
def insertIntoNatTable(dictColNames,table):
    con = duckdb.connect('nationalities.duckdb')

    for row in table.index:

        sql_insert_year = 'INSERT INTO nationalities VALUES ('
        for key in dictColNames:
            if key in table.columns:
                if key == 'Code' or key =='Madrid' or key=='Year':
                    var_char_values='\'{}\''.format(table[key][row])
                    sql_insert_year += f'{var_char_values} ,'
                else:                 
                    sql_insert_year += f'{table[key][row]} ,'

            else:
                sql_insert_year += f' NULL ,'
        
        sql_insert_year=sql_insert_year[0:-1]
        sql_insert_year += ');'
        # print(sql_insert_year)
    
        con.execute(sql_insert_year)
        con.fetchall()
    con.close()

In [98]:

def join_nationalities():
    con = duckdb.connect('nationalities.duckdb')
    
    # display(column_names_2018)
    # display(column_names_2019)
    # display(column_names_2020)

    nat2018 = con.execute("SELECT * FROM nationalities2018").df()
    natcols_2018 = nat2018.columns

    nat2019 = con.execute("SELECT * FROM nationalities2019").df()
    natcols_2019 = nat2019.columns 
    
    nat2020 = con.execute("SELECT * FROM nationalities2020").df()
    natcols_2020 = nat2020.columns 

    # dif1819 = natcols_2018.difference(natcols_2019)
    # dif1918 = natcols_2019.difference(natcols_2018)
    # dif1920 = natcols_2019.difference(natcols_2020)
    # dif2019= natcols_2020.difference(natcols_2019)
    # dif2018= natcols_2020.difference(natcols_2018)
    # dif1820= natcols_2018.difference(natcols_2020)

    # print('2018 difference 2019 (column_name - nationalities)')
    # display(dif1819)
    # print('2019 difference 2018 (column_name - nationalities)')
    # display(dif1918)

    # print('__________________________________________________________')
    # print('2019 difference 2020 (column_name - nationalities)')
    # display(dif1920)
    # print('2020 difference 2019 (column_name - nationalities)')
    # display(dif2019)


    # print('__________________________________________________________')
    # print('2020 difference 2018 (column_name - nationalities)')
    # display(dif2018)
    # print('2018 difference 2020 (column_name - nationalities)')
    # display(dif1820)

    # diff181920 = ((((dif1819.union(dif1920)).union(dif2018)).union(dif2019)).union(dif1820)).union(dif1918)
    # print(diff181920)

    # TO DO - ETL:  Bielorrusia and Belarús (duplicate) etc.

    union181920 = (natcols_2018.union(natcols_2019)).union(natcols_2020)
   
    col_names = union181920.tolist()    
    # print(col_names)
    # print(len(col_names))

    dataTypes = []
    for item in col_names:
        #print(item)
        if item != 'Madrid_section' and item !='Year':
            dataTypes.append('DOUBLE')
        else : 
            dataTypes.append('VARCHAR')
            
    #print(dataTypes)

    dictColNames = dict(zip(col_names, dataTypes))
    # print(dictColNames)

    createNatTable(dictColNames)
    insertIntoNatTable(dictColNames,nat2018)
    insertIntoNatTable(dictColNames,nat2019)
    insertIntoNatTable(dictColNames,nat2020)
    con.close()

In [99]:
 
def trusted_zone():
    join_household()
    join_nationalities()
    

In [100]:
trusted_zone()

,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,Ciudad de Madrid,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0,2018
1,01. Centro,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0,2018
2,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0,2018
3,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0,2018
4,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,21029.0,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0,2020
7391,21030.0,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0,2020
7392,21031.0,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0,2020
7393,21032.0,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0,2020


### Verify joined tables per data source

#### nationalities 

In [101]:
# nationalities joined
con = duckdb.connect('nationalities.duckdb')
view_nationalities = con.execute(f'SELECT * FROM nationalities').df()
display(view_nationalities)
con.close()

,Afganistán,Albania,Alemania,Andorra,Angola,Antigua_y_Barbuda,Arabia_Saudita,Argelia,Argentina,Armenia,...,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,year,Yemen,Zambia,Zimbabwe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0.0,0.0
3,0.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,3.0,0.0,0.0,14.0,0.0,2018,0.0,0.0,1.0
4,0.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,3.0,0.0,0.0,14.0,0.0,2018,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2020,0.0,0.0,0.0
14519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2020,0.0,0.0,0.0
14520,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2020,0.0,0.0,0.0
14521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2020,0.0,0.0,0.0


#### household

In [102]:
# households joined
con = duckdb.connect('household.duckdb')
view_household = con.execute(f'SELECT * FROM household').df()
display(view_household)
con.close()

,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,Ciudad de Madrid,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0,2018
1,01. Centro,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0,2018
2,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0,2018
3,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0,2018
4,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,21029.0,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0,2020
7391,21030.0,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0,2020
7392,21031.0,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0,2020
7393,21032.0,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0,2020
